In [97]:
# Week 1: Data Collection & Exploration
from BRScraper import nba

# running this will show you what metrics are available for each player and their meaning
player_metrics = [
    "Player",  # Name of the player
    "Age",  # Player's age
    "Team",  # Abbreviation of the team the player belongs to
    "Pos (Position)",  # Player's position on the court (e.g., PG, SG, PF)
    "G (Games Played)",  # Number of games played
    "GS (Games Started)",  # Number of games started
    "MP (Minutes Played)",  # Average minutes played per game
    "FG (Field Goals Made)",  # Average field goals made per game
    "FGA (Field Goals Attempted)",  # Average field goal attempts per game
    "FG% (Field Goal Percentage)",  # Percentage of successful field goals
    "3P (Three-Point Field Goals Made)",  # Average three-pointers made per game
    "3PA (Three-Point Field Goals Attempted)",  # Average three-point attempts per game
    "3P% (Three-Point Percentage)",  # Percentage of successful three-pointers
    "2P (Two-Point Field Goals Made)",  # Average two-pointers made per game
    "2PA (Two-Point Field Goals Attempted)",  # Average two-point attempts per game
    "2P% (Two-Point Percentage)",  # Percentage of successful two-pointers
    "eFG% (Effective Field Goal Percentage)",  # Adjusted FG% accounting for 3P being worth more than 2P
    "FT (Free Throws Made)",  # Average free throws made per game
    "FTA (Free Throws Attempted)",  # Average free throw attempts per game
    "FT% (Free Throw Percentage)",  # Percentage of successful free throws
    "ORB (Offensive Rebounds)",  # Average offensive rebounds per game
    "DRB (Defensive Rebounds)",  # Average defensive rebounds per game
    "TRB (Total Rebounds)",  # Average total rebounds per game (ORB + DRB)
    "AST (Assists)",  # Average assists per game
    "STL (Steals)",  # Average steals per game
    "BLK (Blocks)",  # Average blocks per game
    "TOV (Turnovers)",  # Average turnovers per game
    "PF (Personal Fouls)",  # Average personal fouls per game
    "PTS (Points)",  # Average points scored per game
    "Awards (Awards and Rankings)",  # Achievements, awards, and rankings (e.g., MVP-1, AS)
    "Season (Season Year)",  # The season the stats belong to (e.g., 2022-23)
]

for metric in player_metrics:
    print(metric)

Player
Age
Team
Pos (Position)
G (Games Played)
GS (Games Started)
MP (Minutes Played)
FG (Field Goals Made)
FGA (Field Goals Attempted)
FG% (Field Goal Percentage)
3P (Three-Point Field Goals Made)
3PA (Three-Point Field Goals Attempted)
3P% (Three-Point Percentage)
2P (Two-Point Field Goals Made)
2PA (Two-Point Field Goals Attempted)
2P% (Two-Point Percentage)
eFG% (Effective Field Goal Percentage)
FT (Free Throws Made)
FTA (Free Throws Attempted)
FT% (Free Throw Percentage)
ORB (Offensive Rebounds)
DRB (Defensive Rebounds)
TRB (Total Rebounds)
AST (Assists)
STL (Steals)
BLK (Blocks)
TOV (Turnovers)
PF (Personal Fouls)
PTS (Points)
Awards (Awards and Rankings)
Season (Season Year)


In [98]:
print(df.columns)

# checking and handling missing values if neccessary
df.isnull().sum()

print(df.duplicated().sum()) # output is 0, no duplicates were found as expected

print(df.describe())

Index(['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Awards',
       'Season'],
      dtype='object')
0
              Age           G          GS          MP          FG         FGA  \
count  735.000000  735.000000  735.000000  735.000000  735.000000  735.000000   
mean    26.155102   40.880272   18.440816   18.264762    2.972789    6.423401   
std      4.371745   25.581293   25.024108    9.735524    2.356999    4.783384   
min     19.000000    1.000000    0.000000    0.500000    0.000000    0.000000   
25%     23.000000   17.000000    0.000000   10.600000    1.300000    3.000000   
50%     25.000000   41.000000    5.000000   17.200000    2.300000    5.000000   
75%     29.000000   64.500000   29.000000   26.450000    4.150000    8.700000   
max     39.000000   84.000000   82.000000   37.800000   11.500000   23.600000   

In [99]:
# All awards and nominations that can be given to a player during and after the season
awards = {
    'MVP': 'Most Valuable Player',
    'DPOY': 'Defensive Player of the Year',
    'CPOY': 'Comeback Player of the Year',
    'AS': 'All-Star',
    'NBA1': 'All-NBA First Team',
    'NBA2': 'All-NBA Second Team',
    'NBA3': 'All-NBA Third Team',
}

for award in awards:
    print(award)

# for any awards that have a hyphen and a # after (e.g., MVP-6) tells us the player's rank in the voting for that awards in that given season.
# So if a player has MVP-1, they ranked in 1st place and won MVP for that season, if a player has MVP-6, they ranked 6th in MVP voting.

MVP
DPOY
CPOY
AS
NBA1
NBA2
NBA3


In [100]:
df = nba.get_stats(season=2024, info='per_game', playoffs=False)

print(df.head(20)) # getting all metrics/averages for players during the 2022-23 regular season 

                     Player   Age Team Pos     G    GS    MP    FG   FGA  \
0               Joel Embiid  29.0  PHI   C  39.0  39.0  33.6  11.5  21.8   
1               Luka Dončić  24.0  DAL  PG  70.0  70.0  37.5  11.5  23.6   
2     Giannis Antetokounmpo  29.0  MIL  PF  73.0  73.0  35.2  11.5  18.8   
3   Shai Gilgeous-Alexander  25.0  OKC  PG  75.0  75.0  34.0  10.6  19.8   
4             Jalen Brunson  27.0  NYK  PG  77.0  77.0  35.4  10.3  21.4   
5              Devin Booker  27.0  PHO  PG  68.0  68.0  36.0   9.4  19.2   
6              Kevin Durant  35.0  PHO  PF  75.0  75.0  37.2  10.0  19.1   
7              Jayson Tatum  25.0  BOS  PF  74.0  74.0  35.7   9.1  19.3   
8              De'Aaron Fox  26.0  SAC  PG  74.0  74.0  35.9   9.7  20.9   
9          Donovan Mitchell  27.0  CLE  SG  55.0  55.0  35.3   9.1  19.8   
10            Stephen Curry  35.0  GSW  PG  74.0  74.0  32.7   8.8  19.5   
11             Nikola Jokić  28.0  DEN   C  79.0  79.0  34.6  10.4  17.9   
12          